### Using pyds9 and pyregion to do simple photometry on s2222 qso host galaxy.

In [7]:
import numpy as np
from astropy.io import fits
from astropy.io.fits import getdata
import pyds9
import pyregion

In [59]:
dir = "/Volumes/Apps_and_Docs/jrrigby1/Dropbox/SGAS-shared/S2222/Src_plane/"
image_files = ("crop1_test11_Aa_s01_out.fits", "crop2_test22_Aa_s01_out.fits", "crop3_test33_Aa_s01_out.fits")
filters = ("F814W", "F606W", "F436W")

print "Simple aperture photometry"
print "image                        filter   flux_in_aperture  flux_in_bkg   net_flux"

for ii, image in enumerate(image_files) :
    f = fits.open(dir + image)
    dum = f[0].data  # screwing around to get hdu as pyregions wants it
    im = dum.astype(np.float64)
    #im +=0.00 # im is a numpy array, so we can do math on it.
    d = pyds9.DS9('ds9')  # start ds9.  'd' is the way to call ds9
    d.set_np2arr(im) # sending ndarray im directly to ds9
    d.set("zoom to fit")
    
    bad  = pyregion.open(dir + "mask_region2.reg")
    src  = pyregion.open(dir + "bigpoly.reg")
    bkg  = pyregion.open(dir + "box_backgrounds2.reg")
    gross_flux = np.sum(src.get_mask(f[0]) * im) - np.sum(bad.get_mask(f[0]) * im)
    Npix = src.get_mask(f[0]).sum() - bad.get_mask(f[0]).sum()
    bkg_flux = np.median(im[bkg.get_mask(f[0])]) * Npix
    print image, filters[ii], gross_flux, bkg_flux, gross_flux - bkg_flux

Simple aperture photometry
image                        filter   flux_in_aperture  flux_in_bkg   net_flux
crop1_test11_Aa_s01_out.fits F814W 1.41657587677 0.249621147579 1.16695472919
crop2_test22_Aa_s01_out.fits F606W 1.80349407305 0.251288323241 1.55220574981
crop3_test33_Aa_s01_out.fits F436W 0.453654580127 0.078351113063 0.375303467064
